In [1]:
# 📦 Step 1: Install dependencies
!pip install ultralytics roboflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [2]:
# 🔑 Step 2: Authenticate Roboflow and download dataset
from roboflow import Roboflow
rf = Roboflow(api_key="Gj8xhF2AnM1mDaya6bh3")  # <-- Replace with your Roboflow API key
project = rf.workspace("simon-kissing-festq").project("household-items-1381x")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Household-items-1 in yolov11:: 100%|██████████| 15564/15564 [00:03<00:00, 4702.93it/s]


In [3]:
# 🧠 Step 3: Train additively from base yolov11n.pt
!yolo task=detect mode=train \
    model=yolo11m.pt \
    data=/content/Household-items-1/data.yaml \
    epochs=20 \
    batch=16 \
    imgsz=640 \
    lr0=0.001 \
    name=kitchen-finetune \
    patience=10 \
    device=0

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.173 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Household-items-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj

In [4]:
# 📊 Step 4: Validate the trained model
!yolo task=detect mode=val \
    model=runs/detect/kitchen-finetune/weights/best.pt \
    data=/content/Household-items-1/data.yaml \
    split=test

Ultralytics 8.3.173 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 967.4±274.8 MB/s, size: 48.1 KB)
val: Scanning /content/Household-items-1/test/labels... 777 images, 11 backgrounds, 0 corrupt: 100% 777/777 [00:00<00:00, 1750.12it/s]
val: New cache created: /content/Household-items-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 49/49 [00:22<00:00,  2.15it/s]
                   all        777       2085      0.656      0.521      0.585       0.42
                   bed        107        124       0.67      0.516      0.577      0.376
                 chair        215        555      0.669      0.324      0.419      0.272
                 couch        119        155      0.663      0.432      0.514      0.363
          dining table        177        217      0.651      0.

In [5]:
# 📈 Step 6: Generate and show confusion matrix
from ultralytics import YOLO
import matplotlib.pyplot as plt

model = YOLO('runs/detect/kitchen-finetune4/weights/best.pt')
metrics = model.val(data=f"{dataset.location}/data.yaml")
cm = metrics.confusion_matrix

plt.figure(figsize=(8, 6))
cm.plot()
plt.title("Confusion Matrix")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/kitchen-finetune4/weights/best.pt'

In [ ]:
# 📁 Step 5: Optionally save model to Google Drive
from google.colab import drive
drive.mount('/content/drive')
!cp runs/detect/kitchen-finetune/weights/best.pt /content/drive/MyDrive/